In [5]:
import warnings
warnings.filterwarnings('ignore')

In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from amplpy import AMPL, ampl_notebook

In [84]:
ampl = ampl_notebook(
    modules=['coin'],
    license_uuid='default'
)

Exception: Failed to install modules.

In [37]:
TEMP_PATH = r"C:\\Personal\\1. Learning\\portfolio-optimization\\jansen"
idx = pd.IndexSlice
df = pd.read_csv(TEMP_PATH + r"/sp500_2003_2023.csv")
df['date'] = pd.to_datetime(df['date'])
df  = df.set_index('date')

In [38]:
price = ['Close']
cols = df.columns
df.columns = pd.MultiIndex.from_product([cols, price], names=['Ticker','Close'])

In [39]:
sample = df.loc["2013":,:].sample(20, axis=1)

In [77]:
lookback = 252
returns = sample.loc[:,idx[:,'Close']] / sample.loc[:,idx[:,'Close']].shift(1) 
returns = returns.rename(columns={'Close':'returns'})
means = returns.rolling(lookback).mean().rename(columns={'returns':'means'})
var = returns.rolling(lookback).var().rename(columns={'returns':'var'})
data = pd.concat([returns, means, var], axis=1)

n = returns.shape[1]
cov = returns.rolling(lookback).cov().droplevel(2, axis=0).droplevel(1, axis=1)

f = np.zeros(returns.shape)
f[:] = np.nan
g = np.zeros([returns.shape[0]])
g[:] = np.nan

In [72]:
for i, (a, row) in enumerate(data.iterrows()):
    if i < lookback:
        continue
    returns_sub = row.loc[idx[:,'returns']]
    means_sub = row.loc[idx[:,'means']]
    var_sub = row.loc[idx[:,'var']]

    cov_sub = cov.loc[idx[means_sub.name,:],:]
    np.fill_diagonal(cov_sub.values, var_sub)

    means_dict = means_sub.to_dict()
    cov_dict = cov_sub.droplevel(0, axis=0).to_dict()
    cov_dict = {(k1, k2): v for k1 in cov_dict for k2, v in cov_dict[k1].items()}

    ampl.set['ASSETS'] = means_dict.keys()
    ampl.param['returns'] = means_dict
    ampl.param['covars'] = cov_dict
    ampl.param['rfr'] = rfr

    ampl.option['solver'] = "ipopt"
    ampl.solve()

    g[i] = ampl.obj['growth'].value
    f[i] = ampl.var['f'].to_dict().values()
    
